In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-mar-2022/train.csv")
df.head()

In [ ]:
df = df.drop('row_id',axis =1)

In [ ]:
df['direction'].value_counts()

In [ ]:
cat_df = df[['direction']]
df = df.drop(['direction'],axis =1)

**Encoding Direction with Ordinal Encoder**

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
cat_df = ordinal_encoder.fit_transform(cat_df)
cat_df = pd.DataFrame(cat_df,columns=['direction'])

In [ ]:
df = pd.concat([df,cat_df], axis=1)

In [ ]:
df['time'] = pd.to_datetime(df['time'])

In [ ]:
df['day'] = df['time'].dt.day
df['month'] = df['time'].dt.month
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute
df['second'] = df['time'].dt.second

In [ ]:
df = df.drop(['time'],axis=1)

In [ ]:
y_train = df['congestion']
x_train = df.drop(['congestion'],axis=1)

In [ ]:
x_train.hist(bins=50, figsize=(20,15))

**Second has only one value, so we wil drop second**

In [ ]:
x_train = x_train.drop(['second'],axis=1)

In [ ]:
x_train.info()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train)

In [ ]:
from tensorflow.keras import callbacks
early_stopping_cb = callbacks.EarlyStopping(monitor="loss", patience=3)


In [ ]:
import keras
from keras.layers import Dense
from keras.models import Sequential

model = Sequential()
model.add(Dense(10, input_dim=7, kernel_initializer='he_normal', activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='relu'))
model.summary()
model.compile(loss='mse', optimizer='adam', metrics=['mse'])


In [ ]:
model_history = model.fit(x_train, y_train, epochs = 30,callbacks=[early_stopping_cb])

# Pre processing test data

In [ ]:
x_test = pd.read_csv("../input/tabular-playground-series-mar-2022/test.csv")
x_test.head()

In [ ]:
cat_test = x_test[['direction']]
x_test = x_test.drop(['direction'],axis =1)
cat_test = ordinal_encoder.transform(cat_test)
cat_test = pd.DataFrame(cat_test,columns=['direction'])
x_test = pd.concat([x_test,cat_test], axis=1)

In [ ]:
row_id = x_test['row_id']

In [ ]:
x_test['time'] = pd.to_datetime(x_test['time'])
x_test['day'] = x_test['time'].dt.day
x_test['month'] = x_test['time'].dt.month
x_test['hour'] = x_test['time'].dt.hour
x_test['minute'] = x_test['time'].dt.minute
x_test = x_test.drop(['time','row_id'],axis=1)

In [ ]:
x_test.info()

In [ ]:
x_test = scaler.transform(x_test)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
predictions = pd.DataFrame({'pred':predictions[:,0]})

In [ ]:
predictions = predictions['pred'].round(0) 

In [ ]:
output = pd.DataFrame({'row_id': row_id, 'congestion': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")